 step 1 : load the contracts 

In [26]:
import os

contracts_path = r"C:\Users\hp\Downloads\clever contact\clevercontracts/Contracts/2MAINS"

contracts = {}
for filename in os.listdir(contracts_path):
    if filename.endswith(".txt"):  
        with open(os.path.join(contracts_path, filename), "r", encoding="utf-8") as file:
            contracts[filename] = file.read()


print(f"Loaded files: {list(contracts.keys())}")


Loaded files: ['CCAP_2MAINS_BPM031594_vF.txt', 'CCTP_2MAINS_BPM031594_vF.txt', 'cleaned_contract.txt', 'processed_contract1.txt', 'processed_contract11.txt', 'RC 2MAINS BPM031594 vF2.txt']


clean the text files

In [27]:
import re

def clean_text(text):
    # Remove extra line breaks within paragraphs
    text = re.sub(r"(?<!\.\n)\n", " ", text)  # Replace newlines not following a period
    # Normalize multiple spaces
    text = re.sub(r"\s{2,}", " ", text)
    # Replace special characters
    text = text.replace("–", "-").replace("\xa0", " ").replace("•", "-")
    return text
def remove_context_page(text):
    # Remove table of contents and lines with repeated dots
    pattern = r"(Table des matières.*?)(?=Article\s+1\s+-|Articl e\s+1\s+-)"  # Match everything from "Table des matières" to "Article 1 -"
    cleaned_text = re.sub(pattern, "", text, flags=re.DOTALL)
    # Remove lines with "......" and small words around them
    cleaned_text = re.sub(r"(\w+\s*)?\.+(\s*\w+)?", "", cleaned_text)  # Remove lines with dots and surrounding words
    return cleaned_text
# Apply cleaning to all contracts
cleaned_contracts = {filename: remove_context_page(clean_text(content)) for filename, content in contracts.items()}


# Display a sample of the cleaned text
for filename, content in cleaned_contracts.items():
    print(f"Preview of {filename}:")
    print(content[:500])  # Print the first 500 characters
    break
# Save the cleaned content for the first contract
output_path = r"C:\Users\hp\Downloads\clever contact\clevercontracts/Contracts/2MAINS/cleaned_contract.txt"
first_filename = list(cleaned_contracts.keys())[0]
with open(output_path, "w", encoding="utf-8") as file:
    file.write(cleaned_contracts[first_filename])

print(f"The cleaned contract has been saved to {output_path}")

Preview of CCAP_2MAINS_BPM031594_vF.txt:
CCAP_BPM031594 _2MAINS BPM031594 1/39 Secrétariat général Direction des finances, des achats et des services Prestations de développement et de maintenance intégrées d ’application des Ministères sociaux - "2MAINS" Cahier des clauses administratives Numéro de consultation : BPM 031594 Procédure de passation : Appel d ’offres ouvert de l’article  -2 1° du code de la commande publique, CCAP_BPM031594 _2MAINS BPM031594 2/39 Article 1 - PREAMBULE -     Article 2 -      Article 3 - OBJET DE L’ACCORD 
The cleaned contract has been saved to C:\Users\hp\Downloads\clever contact\clevercontracts/Contracts/2MAINS/cleaned_contract.txt


First approach of camamenbert using the preprocessed contract version 1

In [ ]:
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, pipeline
import spacy

# Load the NER pipeline with Camembert
model_checkpoint = "Jean-Baptiste/camembert-ner"
tokenizer = CamembertTokenizerFast.from_pretrained(model_checkpoint)
model = CamembertForTokenClassification.from_pretrained(model_checkpoint)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, framework="pt")

# Load spaCy for sentence segmentation
nlp = spacy.load("fr_core_news_sm")

def sentence_split(text):
    """Split text into sentences using spaCy."""
    doc = nlp(text)
    return [sent.text for sent in doc.sents]

def group_sentences_to_chunks(sentences, tokenizer, max_tokens=512):
    """Group sentences into chunks respecting max token limit."""
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        #count the nb of tokens in each sentence
        tokens = tokenizer(sentence, return_tensors="pt",truncation=True, max_length=max_tokens)["input_ids"].size(1)
        if current_tokens + tokens > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_tokens = 0
        current_chunk.append(sentence)
        current_tokens += tokens

    # Append the last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def split_text_into_chunks(text, tokenizer, max_tokens=128):
    """Split text into smaller chunks at token level."""
    tokens = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True, max_length=max_tokens)
    input_ids = tokens["input_ids"][0].tolist()
    #token_strings = tokenizer.convert_ids_to_tokens(input_ids)
    offsets = tokens["offset_mapping"][0].tolist()

    chunks = []
    current_chunk = []
    current_length = 0

    for idx, token_id in enumerate(input_ids):
        current_chunk.append(token_id)
        current_length += 1

        if current_length >= max_tokens:
            chunks.append(current_chunk)
            current_chunk = []
            current_length = 0

    if current_chunk:
        chunks.append(current_chunk)

    decoded_chunks = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]
    return decoded_chunks

def process_text_with_chunks(text, ner_pipeline, tokenizer, max_tokens=512):
    """Process text by combining sentence and token-level chunking."""
    # Step 1: Sentence-level chunking
    sentences = sentence_split(text)
    sentence_chunks = group_sentences_to_chunks(sentences, tokenizer, max_tokens)

    # Step 2:  token-level chunking
    final_chunks = []
    for chunk in sentence_chunks:
        token_chunks = split_text_into_chunks(chunk, tokenizer, max_tokens)
        final_chunks.extend(token_chunks)

    # Step 3: Process chunks with NER pipeline
    all_predictions = []
    for chunk in final_chunks:
        if len(tokenizer(chunk)["input_ids"]) > max_tokens:
            chunk = tokenizer.decode(tokenizer(chunk)["input_ids"][:max_tokens], skip_special_tokens=True)
        predictions = ner_pipeline(chunk)
        all_predictions.extend(predictions)

    return all_predictions

def group_entities(entities,allowed_types={"B-ORG","I-ORG" ,"B-LOC","I-LOC"} ):
    """Group and consolidate entity predictions."""
    grouped_entities = []
    current_entity = None

    for entity in entities:
        entity_type = entity["entity"]
        word = entity["word"].replace("▁", "")
        start = entity.get("start", 0)
        end = entity.get("end", 0)
        #if entity_type not in allowed_types:
           # continue
        if not any(allowed in entity_type for allowed in allowed_types):
            continue

        if current_entity and current_entity["entity"] == entity_type:
            current_entity["text"] += " " + word
            current_entity["end"] = end
        else:
            if current_entity:
                grouped_entities.append(current_entity)
            current_entity = {"entity": entity_type, "text": word, "start": start, "end": end}

    if current_entity:
        grouped_entities.append(current_entity)

    return grouped_entities


Device set to use cpu


In [20]:
allowed_types = {"B-ORG","I-ORG" ,"B-LOC","I-LOC"} 

# Get the first cleaned contract
first_filename = list(cleaned_contracts.keys())[0]
first_cleaned_contract = cleaned_contracts[first_filename]

# Process the first cleaned contract using the NER pipeline
print(f"Processing first cleaned contract: {first_filename}")

# Apply NER pipeline with chunking
all_predictions = process_text_with_chunks(first_cleaned_contract, ner_pipeline, tokenizer, max_tokens=128)

# Group the entities
grouped_entities = group_entities(all_predictions, allowed_types=allowed_types)

# Display the grouped entities for the first contract
print(f"\nEntities in {first_filename}:")
for entity in grouped_entities:
    print(entity)

Processing first cleaned contract: CCAP_2MAINS_BPM031594_vF.txt

Entities in CCAP_2MAINS_BPM031594_vF.txt:
{'entity': 'I-ORG', 'text': 'Ministère s', 'start': 186, 'end': 197}
{'entity': 'I-LOC', 'text': 'B PM 03 15 B PM Paris 07 SP', 'start': 6, 'end': 11}
{'entity': 'I-ORG', 'text': ' DF AS Secrétariat Général', 'start': 31, 'end': 118}
{'entity': 'I-LOC', 'text': 'Ministère s', 'start': 341, 'end': 352}
{'entity': 'I-ORG', 'text': 'Ministère s Soci aux Ministère s Soci aux Min ist  ères Soci aux', 'start': 313, 'end': 23}
{'entity': 'I-LOC', 'text': 'Ministère s Ministère des Solidarité s et de la Santé Ministère du Travail , de l Emploi Lot 1 Lot 2 Lot 3 Lot 4 Lot Lot Lot Lot Cadre Tour Mira beau D N UM / Fa brique du Numérique Tour Mira beau A ga dir', 'start': 42, 'end': 361}
{'entity': 'I-ORG', 'text': 'Ministère s Soci aux MAIN S MAIN S person ne l', 'start': 281, 'end': 41}
{'entity': 'I-LOC', 'text': 'Les', 'start': 0, 'end': 3}
{'entity': 'I-ORG', 'text': 'publi q  ue Secrét

Same approach of camembert but with the final version of the contract

In [1]:
import json
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, pipeline
import spacy

# Load the NER pipeline with Camembert
model_checkpoint = "Jean-Baptiste/camembert-ner"
tokenizer = CamembertTokenizerFast.from_pretrained(model_checkpoint)
model = CamembertForTokenClassification.from_pretrained(model_checkpoint)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, framework="pt")

# Load spaCy for sentence segmentation
nlp = spacy.load("fr_core_news_sm")

def sentence_split(text):
    """Split text into sentences using spaCy."""
    doc = nlp(text)
    return [sent.text for sent in doc.sents]

def group_sentences_to_chunks(sentences, tokenizer, max_tokens=512):
    """Group sentences into chunks respecting max token limit."""
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        #count the nb of tokens in each sentence
        tokens = tokenizer(sentence, return_tensors="pt",truncation=True, max_length=max_tokens)["input_ids"].size(1)
        if current_tokens + tokens > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_tokens = 0
        current_chunk.append(sentence)
        current_tokens += tokens

    # Append the last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def split_text_into_chunks(text, tokenizer, max_tokens=128):
    """Split text into smaller chunks at token level."""
    tokens = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True, max_length=max_tokens)
    input_ids = tokens["input_ids"][0].tolist()
    #token_strings = tokenizer.convert_ids_to_tokens(input_ids)
    offsets = tokens["offset_mapping"][0].tolist()

    chunks = []
    current_chunk = []
    current_length = 0

    for idx, token_id in enumerate(input_ids):
        current_chunk.append(token_id)
        current_length += 1

        if current_length >= max_tokens:
            chunks.append(current_chunk)
            current_chunk = []
            current_length = 0

    if current_chunk:
        chunks.append(current_chunk)

    decoded_chunks = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]
    return decoded_chunks

def process_text_with_chunks(text, ner_pipeline, tokenizer, max_tokens=512):
    """Process text by combining sentence and token-level chunking."""
    # Step 1: Sentence-level chunking
    sentences = sentence_split(text)
    sentence_chunks = group_sentences_to_chunks(sentences, tokenizer, max_tokens)

    # Step 2:  token-level chunking
    final_chunks = []
    for chunk in sentence_chunks:
        token_chunks = split_text_into_chunks(chunk, tokenizer, max_tokens)
        final_chunks.extend(token_chunks)

    # Step 3: Process chunks with NER pipeline
    all_predictions = []
    for chunk in final_chunks:
        if len(tokenizer(chunk)["input_ids"]) > max_tokens:
            chunk = tokenizer.decode(tokenizer(chunk)["input_ids"][:max_tokens], skip_special_tokens=True)
        predictions = ner_pipeline(chunk)
        all_predictions.extend(predictions)

    return all_predictions

def group_entities(entities, allowed_types={"B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-PER", "I-PER"}):
    """Group and consolidate entity predictions with reformatted output."""
    grouped_entities = []
    current_entity = None

    for entity in entities:
        entity_type = entity["entity"]
        word = entity["word"].replace("▁", "")
        start = entity.get("start", 0)
        end = entity.get("end", 0)

        if not any(allowed in entity_type for allowed in allowed_types):
            continue

        # Clean type from B- or I-
        short_type = entity_type.split("-")[-1]

        if current_entity and current_entity["type"] == short_type:
            current_entity["text"] += " " + word
            current_entity["end"] = end
        else:
            if current_entity:
                grouped_entities.append(current_entity)
            current_entity = {
                "text": word,
                "type": short_type,
                "start": start,
                "end": end
            }

    if current_entity:
        grouped_entities.append(current_entity)

    return grouped_entities


allowed_types = {"B-ORG","I-ORG" ,"B-LOC","I-LOC","B-PER","I-PER"} 
#input_path = r"C:\Users\hp\Downloads\clever contact/cleaned_contract_fixed_final.txt"
#input_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract2.txt"
#input_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract3.txt"
input_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract4.txt"

with open(input_path, "r", encoding="utf-8") as file:
    first_cleaned_contract = file.read()


# Apply NER pipeline with chunking
all_predictions = process_text_with_chunks(first_cleaned_contract, ner_pipeline, tokenizer, max_tokens=128)

# Group the entities
grouped_entities = group_entities(all_predictions, allowed_types=allowed_types)


# Define output filename (you can include the contract name for clarity)
output_filename = f"camembert_firstapproach_entities4.json"

# Save grouped entities to JSON
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(grouped_entities, f, ensure_ascii=False, indent=2)

print(f"\nEntities saved to: {output_filename}")


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu



Entities saved to: camembert_firstapproach_entities4.json


In [21]:
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, pipeline
import spacy

# Load the NER pipeline with Camembert
model_checkpoint = "Jean-Baptiste/camembert-ner"
tokenizer = CamembertTokenizerFast.from_pretrained(model_checkpoint)
model = CamembertForTokenClassification.from_pretrained(model_checkpoint)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, framework="pt")

# Load spaCy for sentence segmentation
nlp = spacy.load("fr_core_news_sm")

def sentence_split(text):
    """Split text into sentences using spaCy."""
    doc = nlp(text)
    return [sent.text for sent in doc.sents]

def group_sentences_to_chunks(sentences, tokenizer, max_tokens=512):
    """Group sentences into chunks respecting max token limit."""
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        tokens = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=max_tokens)["input_ids"].size(1)
        if current_tokens + tokens > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_tokens = 0
        current_chunk.append(sentence)
        current_tokens += tokens

    
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def split_text_into_chunks(text, tokenizer, max_tokens=128):
    """Split text into smaller chunks strictly respecting max token limit."""
    tokens = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True, max_length=max_tokens)
    input_ids = tokens["input_ids"][0].tolist()

    chunks = []
    current_chunk = []
    current_length = 0

    for idx, token_id in enumerate(input_ids):
        current_chunk.append(token_id)
        current_length += 1

        if current_length >= max_tokens:
            chunks.append(current_chunk)
            current_chunk = []
            current_length = 0

    if current_chunk:
        chunks.append(current_chunk)

    decoded_chunks = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]
    return decoded_chunks

def process_text_with_chunks(text, ner_pipeline, tokenizer, max_tokens=512):
    """Process text by combining sentence and token-level chunking."""
    # Step 1: Sentence-level chunking
    sentences = sentence_split(text)
    sentence_chunks = group_sentences_to_chunks(sentences, tokenizer, max_tokens)

    # Step 2: Token-level chunking
    final_chunks = []
    for chunk in sentence_chunks:
        token_chunks = split_text_into_chunks(chunk, tokenizer, max_tokens)
        final_chunks.extend(token_chunks)

    # Step 3: Process chunks with NER pipeline
    all_predictions = []
    for chunk in final_chunks:
        predictions = ner_pipeline(chunk)
        all_predictions.extend(predictions)

    return all_predictions


allowed_types = {"B-ORG", "I-ORG", "B-LOC", "I-LOC"}

# Get the first cleaned contract
first_filename = list(cleaned_contracts.keys())[0]
first_cleaned_contract = cleaned_contracts[first_filename]


print(f"Processing first cleaned contract: {first_filename}")
print(f"Text snippet: {first_cleaned_contract[:500]}")  # Show the first 500 characters

all_predictions = process_text_with_chunks(first_cleaned_contract, ner_pipeline, tokenizer, max_tokens=128)

filtered_predictions = [
    prediction for prediction in all_predictions if prediction["entity"] in allowed_types
]


print(f"\nFiltered Entities in {first_filename}:")
for entity in filtered_predictions:
    print(entity)


Device set to use cpu


Processing first cleaned contract: CCAP_2MAINS_BPM031594_vF.txt
Text snippet: CCAP_BPM031594 _2MAINS BPM031594 1/39 Secrétariat général Direction des finances, des achats et des services Prestations de développement et de maintenance intégrées d ’application des Ministères sociaux - "2MAINS" Cahier des clauses administratives Numéro de consultation : BPM 031594 Procédure de passation : Appel d ’offres ouvert de l’article  -2 1° du code de la commande publique, CCAP_BPM031594 _2MAINS BPM031594 2/39 Article 1 - PREAMBULE -     Article 2 -      Article 3 - OBJET DE L’ACCORD 

Filtered Entities in CCAP_2MAINS_BPM031594_vF.txt:
{'entity': 'I-ORG', 'score': np.float32(0.6703275), 'index': 45, 'word': '▁Ministère', 'start': 186, 'end': 196}
{'entity': 'I-ORG', 'score': np.float32(0.78778434), 'index': 46, 'word': 's', 'start': 196, 'end': 197}
{'entity': 'I-LOC', 'score': np.float32(0.7271802), 'index': 4, 'word': '▁B', 'start': 6, 'end': 8}
{'entity': 'I-LOC', 'score': np.float32(0.8108737),

In [2]:
from transformers import AutoTokenizer, CamembertForTokenClassification, pipeline

# Chunking function
def chunk_text_with_overlap(text, tokenizer, max_tokens=128, overlap_tokens=16):
    tokens = tokenizer(text, return_tensors="pt", truncation=False, add_special_tokens=False)
    input_ids = tokens["input_ids"][0].tolist()

    chunks = []
    start = 0

    while start < len(input_ids):
        end = start + max_tokens
        chunk_ids = input_ids[start:end]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append(chunk_text)
        start += max_tokens - overlap_tokens

    return chunks


def process_chunks_with_ner(chunks, ner_pipeline, allowed_types={"B-ORG", "I-ORG", "B-LOC", "I-LOC"}):
    
    all_entities = []

    for chunk in chunks:
        # Apply the NER pipeline
        predictions = ner_pipeline(chunk)

        # Filter predictions by allowed types
        filtered_predictions = [
            {
                "entity": pred["entity"],
                "word": pred["word"].replace("▁", ""),  # Clean up tokenized words
                "start": pred.get("start", None),
                "end": pred.get("end", None),
            }
            for pred in predictions
            if pred["entity"] in allowed_types
        ]

        # Add to the list of entities
        all_entities.extend(filtered_predictions)

    return all_entities




In [3]:
def group_entities(entities, allowed_types={"B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-PER", "I-PER"}):
    """Group and consolidate entity predictions with reformatted output."""
    grouped_entities = []
    current_entity = None

    for entity in entities:
        entity_type = entity["entity"]
        word = entity["word"].replace("▁", "")
        start = entity.get("start", 0)
        end = entity.get("end", 0)

        if not any(allowed in entity_type for allowed in allowed_types):
            continue

        # Clean type from B- or I-
        short_type = entity_type.split("-")[-1]

        if current_entity and current_entity["type"] == short_type:
            current_entity["text"] += " " + word
            current_entity["end"] = end
        else:
            if current_entity:
                grouped_entities.append(current_entity)
            current_entity = {
                "text": word,
                "type": short_type,
                "start": start,
                "end": end
            }

    if current_entity:
        grouped_entities.append(current_entity)

    return grouped_entities



In [30]:
if __name__ == "__main__":
    # Initialize tokenizer and NER model
    model_checkpoint = "Jean-Baptiste/camembert-ner"
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = CamembertForTokenClassification.from_pretrained(model_checkpoint)
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, framework="pt")

    # Assuming cleaned_contracts is already loaded
    first_filename = list(cleaned_contracts.keys())[0]
    first_cleaned_contract = cleaned_contracts[first_filename]

    print(f"Processing first contract: {first_filename}")
    print(f"Text snippet: {first_cleaned_contract[:500]}")  # Display the first 500 characters of the contract text

    # Chunking parameters
    max_tokens = 128
    overlap_tokens = 16

    # Step 1: Chunk the contract
    chunks = chunk_text_with_overlap(first_cleaned_contract, tokenizer, max_tokens=max_tokens, overlap_tokens=overlap_tokens)

    # Step 2: Apply NER pipeline to chunks
    raw_entities = process_chunks_with_ner(chunks, ner_pipeline)

    # Step 3: Group fragmented entities
    grouped_entities = group_entities(raw_entities)

    # Step 4: Display results for the first contract
    print(f"\nGrouped Entities in {first_filename}:")
    for entity in grouped_entities:
        print(entity)


Device set to use cpu


Processing first contract: CCAP_2MAINS_BPM031594_vF.txt
Text snippet: CCAP_BPM031594 _2MAINS BPM031594 1/39 Secrétariat général Direction des finances, des achats et des services Prestations de développement et de maintenance intégrées d ’application des Ministères sociaux - "2MAINS" Cahier des clauses administratives Numéro de consultation : BPM 031594 Procédure de passation : Appel d ’offres ouvert de l’article  -2 1° du code de la commande publique, CCAP_BPM031594 _2MAINS BPM031594 2/39 Article 1 - PREAMBULE -     Article 2 -      Article 3 - OBJET DE L’ACCORD 


Token indices sequence length is longer than the specified maximum sequence length for this model (22844 > 512). Running this sequence through the model will result in indexing errors



Grouped Entities in CCAP_2MAINS_BPM031594_vF.txt:
{'entity': 'I-ORG', 'text': 'Direction Ministère s sociaux', 'start': 57, 'end': 203}
{'entity': 'I-LOC', 'text': 'Sous', 'start': 20, 'end': 25}
{'entity': 'I-ORG', 'text': 'Ministère s sociaux', 'start': 622, 'end': 641}
{'entity': 'I-LOC', 'text': 'Direction des finances', 'start': 641, 'end': 664}
{'entity': 'I-ORG', 'text': 'Ministère s sociaux', 'start': 23, 'end': 42}
{'entity': 'I-LOC', 'text': 'avenue Du ques ne Paris', 'start': 151, 'end': 179}
{'entity': 'I-ORG', 'text': ' DF AS Ministère s sociaux Secrétariat Général Direction du numérique', 'start': 205, 'end': 315}
{'entity': 'I-LOC', 'text': 'Ministère s Ministère s Ministère s Soci aux', 'start': 515, 'end': 372}
{'entity': 'I-ORG', 'text': 'Ministère s Soci aux Min ist  ères Soci aux', 'start': 325, 'end': 439}
{'entity': 'I-LOC', 'text': 'Ministère s', 'start': 458, 'end': 469}
{'entity': 'I-ORG', 'text': 'Min ist  ères Soci aux', 'start': 8, 'end': 28}
{'entity': 'I-

In [31]:
if __name__ == "__main__":
    # Initialize tokenizer and NER model
    model_checkpoint = "Jean-Baptiste/camembert-ner"
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = CamembertForTokenClassification.from_pretrained(model_checkpoint)
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, framework="pt")

    # Assuming cleaned_contracts is already loaded
    first_filename = list(cleaned_contracts.keys())[1]
    first_cleaned_contract = cleaned_contracts[first_filename]

    print(f"Processing first contract: {first_filename}")
    print(f"Text snippet: {first_cleaned_contract[:500]}")  # Display the first 500 characters of the contract text

    # Chunking parameters
    max_tokens = 128
    overlap_tokens = 16

    # Step 1: Chunk the contract
    chunks = chunk_text_with_overlap(first_cleaned_contract, tokenizer, max_tokens=max_tokens, overlap_tokens=overlap_tokens)

    # Step 2: Apply NER pipeline to chunks
    raw_entities = process_chunks_with_ner(chunks, ner_pipeline)

    # Step 3: Group fragmented entities
    grouped_entities = group_entities(raw_entities)

    # Step 4: Display results for the first contract
    print(f"\nGrouped Entities in {first_filename}:")
    for entity in grouped_entities:
        print(entity)


Device set to use cpu


Processing first contract: CCTP_2MAINS_BPM031594_vF.txt
Text snippet: Secrétariat général Direction du numérique 24 janvier 2022 Cahier des Clauses Techniques Particulières Prestations de DEveloppement et de MAintenance INtégrées d’applications des Ministères Sociaux Marché 2MAINS 2 / 180 Table des matières  du      Objet du           Bénéficiaires du      du      Le Secrétariat      La      Missions et activités de la          Le parc      Chiffres clefs du SI des     Hébergement des     du      Un marché unifié et     L’amélioration et l’harmonisation de la qual


Token indices sequence length is longer than the specified maximum sequence length for this model (95130 > 512). Running this sequence through the model will result in indexing errors



Grouped Entities in CCTP_2MAINS_BPM031594_vF.txt:
{'entity': 'I-ORG', 'text': 'Secrétariat Direction du numérique', 'start': 0, 'end': 42}
{'entity': 'I-LOC', 'text': 'Environnement s Ministère s S Ministère s S Ministère', 'start': 145, 'end': 551}
{'entity': 'I-ORG', 'text': 'D N UM na tu re Titulaire Ministère Ministère Ministère', 'start': 51, 'end': 416}
{'entity': 'I-LOC', 'text': 'Ministère Ministère', 'start': 184, 'end': 346}
{'entity': 'I-ORG', 'text': 'D N UM', 'start': 405, 'end': 410}
{'entity': 'I-LOC', 'text': 'Ministère s Soci aux Ministère s Soci aux Gouvernement Ministère s Soci aux', 'start': 165, 'end': 604}
{'entity': 'I-ORG', 'text': 'Ministère s Soci aux', 'start': 18, 'end': 37}
{'entity': 'I-LOC', 'text': 'Ministère s Ministère des Solidarité s et de la Santé Ministère du Travail , de l Emploi', 'start': 69, 'end': 170}
{'entity': 'I-ORG', 'text': 'D N UM Jeunesse Sports D N', 'start': 208, 'end': 459}
{'entity': 'I-LOC', 'text': 'UM', 'start': 459, 'end': 461

Trying the second approach of camembert with the final version of the preproccesed contract

In [4]:
if __name__ == "__main__":
    # Initialize tokenizer and NER model
    model_checkpoint = "Jean-Baptiste/camembert-ner"
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = CamembertForTokenClassification.from_pretrained(model_checkpoint)
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, framework="pt")

    #input_path = r"C:\Users\hp\Downloads\clever contact/cleaned_contract_fixed_final.txt"
    #input_path = r"C:\Users\hp\Downloads\clever contact/cleanedcontract2.txt"
    #input_path = r"C:\Users\hp\Downloads\clever contact/cleanedcontract3.txt"
    input_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract4.txt"
    with open(input_path, "r", encoding="utf-8") as file:
        first_cleaned_contract = file.read()

    

    # Chunking parameters
    max_tokens = 128
    overlap_tokens = 16

    # Step 1: Chunk the contract
    chunks = chunk_text_with_overlap(first_cleaned_contract, tokenizer, max_tokens=max_tokens, overlap_tokens=overlap_tokens)

    # Step 2: Apply NER pipeline to chunks
    raw_entities = process_chunks_with_ner(chunks, ner_pipeline)

    # Step 3: Group fragmented entities
    grouped_entities = group_entities(raw_entities)
    # Define output filename (you can include the contract name for clarity)
    output_filename = f"camembert_second_approach_entities4.json"

# Save grouped entities to JSON
    with open(output_filename, "w", encoding="utf-8") as f:
         json.dump(grouped_entities, f, ensure_ascii=False, indent=2)

    print(f"\nEntities saved to: {output_filename}")
    

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (29066 > 512). Running this sequence through the model will result in indexing errors



Entities saved to: camembert_second_approach_entities4.json
